In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras import Model, Input
# Load the dataset
file_path = '/kaggle/input/final-dengue-dataset/DengueFinalDatasets.csv'
dengue_data = pd.read_csv(file_path)

# Identifying numerical and categorical columns
categorical_cols = dengue_data.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = dengue_data.select_dtypes(include=['number']).columns.tolist()
categorical_cols.remove('NS1')  # Removing the target variable from categorical columns

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define target and predictors
X = dengue_data.drop('NS1', axis=1)
y = dengue_data['NS1']

# Encode target variable
y_encoded = np.where(y == 'Positive', 1, 0)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

# Convert to NumPy arrays and Reshape data for RNN
X_train_rnn = X_train.to_numpy().reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_rnn = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load your dataset
file_path = '/kaggle/input/final-dengue-dataset/DengueFinalDatasets.csv'
dengue_data = pd.read_csv(file_path)

# Identify categorical and numerical columns
categorical_cols = dengue_data.select_dtypes(include=['object']).columns.tolist()
numerical_cols = dengue_data.select_dtypes(include=['number']).columns.tolist()
categorical_cols.remove('NS1')  # Assuming 'NS1' is the target

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Separate features and target variable
X = dengue_data.drop('NS1', axis=1)
y = dengue_data['NS1'].apply(lambda x: 1 if x == 'Positive' else 0)  # Encode the target variable

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Apply preprocessing and convert to dense format
X_train = preprocessor.fit_transform(X_train).toarray()
X_test = preprocessor.transform(X_test).toarray()

# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Reshape the data for RNN input
X_train_rnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_rnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [3]:
# # Hyperparameters
# n_gru_units = 128  # Number of units in GRU layers
# transformer_heads = 50  # Number of attention heads in transformer
# dropout_rate = 0.4
# learning_rate = 0.001

# def build_simplified_model(input_shape, n_gru_units, transformer_heads, dropout_rate):
#     inputs = Input(shape=input_shape)
    
#     # GRU layers
#     x = GRU(n_gru_units, return_sequences=True)(inputs)
#     x = Dropout(dropout_rate)(x)
#     x = GRU(n_gru_units, return_sequences=True)(x)
#     x = Dropout(dropout_rate)(x)

#     # Transformer layer
#     transformer_layer = MultiHeadAttention(num_heads=transformer_heads, key_dim=n_gru_units)
#     x = transformer_layer(x, x)
#     x = LayerNormalization()(x)

#     # Final layers
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(dropout_rate)(x)
#     outputs = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs, outputs)
#     return model

# # Update the input shape based on the preprocessed data
# input_shape = (X_train_rnn.shape[1], 1) 

# # Rebuild the model
# model = build_simplified_model(input_shape, n_gru_units, transformer_heads, dropout_rate)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [4]:
# Hyperparameters
n_gru_units = 128  # Number of units in GRU layers
transformer_heads = 50  # Number of attention heads in transformer
dropout_rate = 0.4
learning_rate = 0.001

def build_complex_model(input_shape, n_gru_units, transformer_heads, dropout_rate):
    inputs = Input(shape=input_shape)
    
    # Multiple GRU layers with increasing complexity
    x = GRU(n_gru_units, return_sequences=True)(inputs)
    x = Dropout(dropout_rate)(x)
    x = GRU(n_gru_units * 2, return_sequences=True)(x)  # Increasing units
    x = Dropout(dropout_rate)(x)
    x = GRU(n_gru_units * 4, return_sequences=True)(x)  # Further increasing units
    x = Dropout(dropout_rate)(x)

    # Adding more transformer layers
    for _ in range(3):  # Three transformer layers
        transformer_layer = MultiHeadAttention(num_heads=transformer_heads, key_dim=n_gru_units * 4)
        x = transformer_layer(x, x)
        x = LayerNormalization()(x)

    # Final layers
    x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

input_shape = (X_train_rnn.shape[1], 1) 
# Rebuild the model with updated parameters
model = build_complex_model(input_shape, n_gru_units, transformer_heads, dropout_rate)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
history = model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_split=0.1)
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_rnn, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Plotting code (as before)


Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node 'model/multi_head_attention_2/value/einsum/Einsum' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_o

In [ ]:


# Optionally, plot the training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()
